단일 값 검색에 최적화된 at과 iat이 존재한다. loc와 iloc에 비해서 속도가 빠르다. 

In [1]:
import numpy as np
import pandas as pd

In [2]:
nba = pd.read_csv('datas/nba.csv', index_col='Name')
nba.nlargest(1, columns=['Salary'])

,Team,Position,Birthday,Salary
Name,,,,
Stephen Curry,Golden State Warriors,PG,3/14/88,40231758


In [4]:
%%timeit
nba.at['Stephen Curry', 'Birthday']

1.97 µs ± 18.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [6]:
%%timeit
nba.loc['Stephen Curry', 'Birthday']

4.13 µs ± 32 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


rename() 메서드로 index를 변경할 수 있따. 키워드 인수에 딕셔너리를 전달하면 된다.

In [7]:
nba = nba.rename(index={'Giannis Antetokounmpo': 'Greek Freak'})

In [8]:
nba.loc['Greek Freak']

Team        Milwaukee Bucks
Position                 PF
Birthday            12/6/94
Salary             25842697
Name: Greek Freak, dtype: object

연습문제

In [ ]:
nfl = pd.read_csv('datas/nfl.csv', index_col='Name', parse_dates=['Birthday'])
nfl

,Team,Position,Birthday,Salary
Name,,,,
Tremon Smith,Philadelphia Eagles,RB,1996-07-20,570000
Shawn Williams,Cincinnati Bengals,SS,1991-05-13,3500000
Adam Butler,New England Patriots,DT,1994-04-12,645000
Derek Wolfe,Denver Broncos,DE,1990-02-24,8000000
Jake Ryan,Jacksonville Jaguars,OLB,1992-02-27,1000000
...,...,...,...,...
Bashaud Breeland,Kansas City Chiefs,CB,1992-01-30,805000
Craig James,Philadelphia Eagles,CB,1996-04-29,570000
Jonotthan Harrison,New York Jets,C,1991-08-25,1500000


연습문제 - 팀당 선수가 몇 명인지 출력해보시오.

In [17]:
nfl.Team.value_counts()

New York Jets           58
Kansas City Chiefs      56
Washington Redskins     56
New Orleans Saints      55
San Francisco 49Ers     55
Denver Broncos          54
Minnesota Vikings       54
Los Angeles Chargers    54
Seattle Seahawks        53
Dallas Cowboys          53
Buffalo Bills           53
Atlanta Falcons         53
Detroit Lions           53
Chicago Bears           53
Los Angeles Rams        52
New York Giants         52
Philadelphia Eagles     52
Houston Texans          52
Arizona Cardinals       51
Cincinnati Bengals      51
Green Bay Packers       51
Oakland Raiders         51
Jacksonville Jaguars    50
Cleveland Browns        49
Miami Dolphins          49
Indianapolis Colts      49
Carolina Panthers       49
New England Patriots    49
Baltimore Ravens        48
Pittsburgh Steelers     47
Tampa Bay Buccaneers    47
Tennessee Titans        46
Name: Team, dtype: int64

In [18]:
nfl.Salary.nlargest(5)

Name
Kirk Cousins       27500000
Marcus Mariota     20922000
Jameis Winston     20922000
Derek Carr         19900000
Jimmy Garoppolo    17200000
Name: Salary, dtype: int64

연습문제 - 팀마다 연봉을 많이 받는 사람들을 내림차순으로 조회해보시오. 팀은 오름차순으로 정렬한다.

In [26]:
nfl.sort_values(['Team', 'Salary'], ascending=[True, False])

,Team,Position,Birthday,Salary
Name,,,,
Chandler Jones,Arizona Cardinals,OLB,1990-02-27,16500000
Patrick Peterson,Arizona Cardinals,CB,1990-07-11,11000000
Larry Fitzgerald,Arizona Cardinals,WR,1983-08-31,11000000
David Johnson,Arizona Cardinals,RB,1991-12-16,5700000
Justin Pugh,Arizona Cardinals,G,1990-08-15,5000000
...,...,...,...,...
Ross Pierschbacher,Washington Redskins,C,1995-05-05,495000
Kelvin Harmon,Washington Redskins,WR,1996-12-15,495000
Wes Martin,Washington Redskins,G,1996-05-09,495000


apply() 메서드는 Series의 각 값에 모두 함수를 적용하는 메서드이다. 첫 번째 인자 값은 소괄호 없는 함수 객체이며 필수값이다.

In [30]:
pokemons = pd.read_csv('datas/pokemon.csv', index_col=['Pokemon']).squeeze()
def nofType(s):
    if '/' in s:
        return 'Multi'
    return 'Single'

pokemons.apply(nofType).value_counts()

Multi     405
Single    404
Name: Type, dtype: int64

데이터프레임에 대해 apply()를 적용할 땐 axis를 명시해줘야 한다.

In [34]:
df = pd.DataFrame([[4, 9]]*3, columns = ['A', 'B'])
df.apply(np.sqrt)

,A,B
0,2.0,3.0
1,2.0,3.0
2,2.0,3.0


np.sqrt()는 ufunc이기 때문에 df를 그대로 인자로 넣어도 작동한다. 

In [37]:
df.apply(lambda x: pd.Series([1, 2], index=['foo', 'bar']), axis=1)

,foo,bar
0,1,2
1,1,2
2,1,2


result_type='broadcast'를 인수로 전달하면 동일한 shape의 결과를 보장한다. 함수로부터 반환되는 것이 리스트인지 스칼라인지에 상관없이 axis 방향으로 브로드캐스트한다. 이때 함수로부터 return되는 값이 기존 shape으로 브로드캐스트할 수 없다면 ValueError가 발생한다. 

예를 들어 column마다의 최댓값과 최솟값의 차이를 구하고 싶다면 다음과 같이 하면 된다.

In [38]:
df3 = pd.DataFrame({'A':[1, 3, 4, 3, 4], 'B':[2, 3, 1, 3, 2], 'C':[1, 5, 2, 4, 4]})
df3.apply(lambda x: x.max()-x.min())

A    3
B    2
C    4
dtype: int64

In [85]:
import seaborn as sns
titanic = sns.load_dataset('titanic')
titanic['adult/child'] = titanic.apply(lambda r: 'adult' if r.age>=20 else 'child', axis=1)
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,adult/child
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,adult
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,adult
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,adult
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,adult
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,adult
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True,adult
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True,child
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False,child
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True,adult


연습문제 - 타이타닉호의 승객에 대해 나이와 성별에 의한 카테고리 column인 category1 열을 만들어 보시오. 

(1) 20살이 넘으면 성별을 그대로 사용한다.

(2) 20살 미만이면 성별에 관계없이 'child'라고 한다.

In [66]:
titanic['category1'] = titanic.apply(lambda r: r.sex if r.age>=20 else 'child', axis=1)
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,adult/child,category1
0,0,3,male,29.699118,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,adult,male
1,1,1,female,29.699118,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,adult,female
2,1,3,female,29.699118,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,adult,female
3,1,1,female,29.699118,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,adult,female
4,0,3,male,29.699118,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,adult,male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,29.699118,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True,adult,male
887,1,1,female,29.699118,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True,child,female
888,0,3,female,29.699118,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False,child,female
889,1,1,male,29.699118,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True,adult,male


fillna() 메서드로 NaN 값을 0으로 변경할 수 있다.

In [ ]:
df = pd.DataFrame([[np.nan, 2, np.nan, 0],
                   [3, 4, np.nan, 1],
                   [np.nan, np.nan, np.nan, np.nan],
                   [np.nan, 3, np.nan, 4]],
                  columns = list("ABCD"))
df.fillna(0)

,A,B,C,D
0,0.0,2.0,0.0,0.0
1,3.0,4.0,0.0,1.0
2,0.0,0.0,0.0,0.0
3,0.0,3.0,0.0,4.0


value 값으로 column label을 key로 갖는 딕셔너리를 전달할 수 있다. 그러면 column마다 NaN을 대체하는 값을 각각 다르게 지정할 수 있다. 또 limit 키워드로 제한된 횟수만큼만 변경할 수 있다.

연습문제 - 타이타닉호 승객 중 나이를 명시하지 않은 고객은 나이를 명시한 고객의 평균 나이값이 되도록 고쳐보시오.

In [91]:
titanic.age.fillna(titanic.age.mean().round(2), inplace=True)
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,adult/child
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,adult
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,adult
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,adult
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,adult
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,adult
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True,adult
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True,child
888,0,3,female,29.7,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False,child
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True,adult


astype() 메서드로 데이터 타입을 바꾸는 것도 가능하다. 첫 인자로 변경해 줄 dtype을 전달하면 된다.

연습문제 - employee.csv는 직원의 이름, 성별, 입사일, 연봉, 관리자 여부, 팀의 열을 갖는다. 이를 데이터프레임으로 불러오시오.

In [122]:
emp = pd.read_csv('datas/employees.csv', parse_dates=['Start Date'])
emp

,First Name,Gender,Start Date,Salary,Mgmt,Team
0,Douglas,Male,1993-08-06,NaN,True,Marketing
1,Thomas,Male,1996-03-31,61933.0,True,NaN
2,Maria,Female,NaT,130590.0,False,Finance
3,Jerry,NaN,2005-03-04,138705.0,True,Finance
4,Larry,Male,1998-01-24,101004.0,True,IT
...,...,...,...,...,...,...
996,Phillip,Male,1984-01-31,42392.0,False,Finance
997,Russell,Male,2013-05-20,96914.0,False,Product
998,Larry,Male,2013-04-20,60500.0,False,Business Dev
999,Albert,Male,2012-05-15,129949.0,True,Sales


In [123]:
emp.Mgmt.fillna(False, inplace=True)

In [124]:
emp.Mgmt.astype(bool)

0        True
1        True
2       False
3        True
4        True
        ...  
996     False
997     False
998     False
999      True
1000    False
Name: Mgmt, Length: 1001, dtype: bool

In [125]:
emp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   First Name  933 non-null    object        
 1   Gender      854 non-null    object        
 2   Start Date  999 non-null    datetime64[ns]
 3   Salary      999 non-null    float64       
 4   Mgmt        1001 non-null   bool          
 5   Team        957 non-null    object        
dtypes: bool(1), datetime64[ns](1), float64(1), object(3)
memory usage: 40.2+ KB


Salary의 결측치가 2개 존재한다. 이를 0으로 변경하고, 소수점 값이 있는지 검토하고 없다면 int형으로 변경한다.

In [ ]:
emp.Salary.fillna(0, inplace=True)
emp.Salary = emp.Salary.apply(lambda x: int(x) if x.is_interger() else x)

In [136]:
emp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   First Name  933 non-null    object        
 1   Gender      854 non-null    object        
 2   Start Date  999 non-null    datetime64[ns]
 3   Salary      1001 non-null   int64         
 4   Mgmt        1001 non-null   bool          
 5   Team        957 non-null    object        
dtypes: bool(1), datetime64[ns](1), int64(1), object(3)
memory usage: 40.2+ KB


연습문제 - 컬럼마다 고유한 값의 숫자가 얼마나 되는지 확인하고, 그 중 범주형으로 만들면 좋을 컬럼이 어떤 것이 있는지 고민해보고 적용해보시오.

In [142]:
emp.Gender = emp.Gender.astype('category')
emp.Team = emp.Team.astype('category')

In [143]:
emp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   First Name  933 non-null    object        
 1   Gender      854 non-null    category      
 2   Start Date  999 non-null    datetime64[ns]
 3   Salary      1001 non-null   int64         
 4   Mgmt        1001 non-null   bool          
 5   Team        957 non-null    category      
dtypes: bool(1), category(2), datetime64[ns](1), int64(1), object(1)
memory usage: 27.0+ KB


연습문제 - 타이타닉호의 승객에 대해 나이와 성별에 의한 카테고리 column인 category2 colum을 만들어보시오.

(1) 성별을 나타내는 문자열 male 또는 female로 시작

(2) 뒤에 나이를 나타내는 문자열이 온다.

In [157]:
titanic['category2'] = titanic.apply(lambda r: r.sex+str(r.age), axis = 1)
titanic.category2 = titanic.category2.astype('category')
titanic.category2.dtype

CategoricalDtype(categories=['female0.75', 'female1.0', 'female10.0', 'female11.0',
                  'female13.0', 'female14.0', 'female14.5', 'female15.0',
                  'female16.0', 'female17.0',
                  ...
                  'male65.0', 'male66.0', 'male7.0', 'male70.0', 'male70.5',
                  'male71.0', 'male74.0', 'male8.0', 'male80.0', 'male9.0'],
, ordered=False)

실수 값을 크기 기준으로 해서 카테고리 값으로 변환하고 싶을 때는 cut 또는 qcut을 사용한다. 

In [158]:
ages = [0, 2, 10, 21, 23, 37, 31, 61, 20, 41, 32, 101]
bins = [1, 20, 30, 50, 70, 100]
labels = ['미성년자', '청년', '장년', '중년', '노년']
cats = pd.cut(ages, bins, labels=labels)
cats

[NaN, '미성년자', '미성년자', '청년', '청년', ..., '중년', '미성년자', '장년', '장년', NaN]
Length: 12
Categories (5, object): ['미성년자' < '청년' < '장년' < '중년' < '노년']

cut 명령이 반환하는 값은 Categorical 클래스 객체이다. 이 객체는 categories 속성으로 label 문자열을 가진다. 따라서 값이 문자열이 아니므로 문자열로 만들려면 astype 메서드를 사용해야 한다. 

qcut 명령은 구간 경계선을 지정하지 않고 분위수와 같이 데이터 개수가 같도록 구간을 나눈다. 예를 들어 다음 코드는 1000개의 데이터를 4개의 구간으로 쪼갠다.

In [159]:
data = np.random.randn(1000)
cats = pd.qcut(data, 4, labels=['q1', 'q2', 'q3', 'q4'])
cats

['q4', 'q4', 'q2', 'q1', 'q4', ..., 'q4', 'q2', 'q3', 'q3', 'q4']
Length: 1000
Categories (4, object): ['q1' < 'q2' < 'q3' < 'q4']

In [160]:
pd.value_counts(cats)

q1    250
q2    250
q3    250
q4    250
dtype: int64

연습문제 - 타이타닉호 승객을 나이 그룹으로 나눈다. 각 나의 그룹의 승객 비율을 구한다. 비율의 전체 합은 1이 되어야 한다.

In [173]:
bins = [1, 20, 30, 50, 70, 100]
labels = ["미성년자", "청년", "장년", "중년", "노년"]
titanic_cats = pd.cut(titanic.age, bins, labels=labels)
titanic_cats.value_counts(normalize=True)
titanic['cat'] = titanic_cats

In [175]:
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,adult/child,category2,category3,cat
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,adult,male22.0,,청년
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,adult,female38.0,,장년
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,adult,female26.0,,청년
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,adult,female35.0,,장년
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,adult,male35.0,,장년
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True,adult,male27.0,,청년
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True,child,female19.0,미성년자,미성년자
888,0,3,female,29.7,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False,child,female29.7,,청년
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True,adult,male26.0,,청년


연습문제 - 타이타닉호의 승객에 대해 나이와 성별에 의한 카테고리 column인 category3을 만들어라.

(1) 20살 미만이면 성별에 관계없이 미성년자 라고 한다.

(2) 20살 이상이면 나이에 따라 청년, 장년, 중년, 노년을 구분하고 그 뒤에 성별을 나타내는 남성, 여성을 붙인다.

In [183]:
titanic['category3'] = titanic_cats
titanic['ksex'] = titanic.apply(lambda x: ' 남성' if x.sex=='male' else ' 여성', axis=1)
titanic.category3 =titanic.apply(lambda x: str(x.category3)+str(x.ksex) if x.category3 != '미성년자' else '미성년자', axis=1)
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,adult/child,category2,category3,cat,ksex
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False,adult,male22.0,청년 남성,청년,남성
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False,adult,female38.0,장년 여성,장년,여성
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True,adult,female26.0,청년 여성,청년,여성
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False,adult,female35.0,장년 여성,장년,여성
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True,adult,male35.0,장년 남성,장년,남성
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True,adult,male27.0,청년 남성,청년,남성
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True,child,female19.0,미성년자,미성년자,여성
888,0,3,female,29.7,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False,child,female29.7,청년 여성,청년,여성
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True,adult,male26.0,청년 남성,청년,남성
